# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [3]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'
#print(ROBOT_POSE_DATA_DIR)

Populating the interactive namespace from numpy and matplotlib


In [4]:
classes = listdir(ROBOT_POSE_DATA_DIR) #listdir schreibt alle file namen von dem path auf, path ist robot_pose_data
print(classes)

['Crouch', 'Belly', 'Knee', 'Frog', 'HeadBack', 'Back', 'Left', 'StandInit', 'Stand', 'Right', 'Sit']


In [98]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE ---hier wird schon data reingelesen??
    #pickle.load benötigt als argument eine Datei
    #hatte mit wb statt rb die Knee Datei gelöscht (lokal) konnte sie mit git checkout --robot_pose_data/Knee wiederherstellen
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    #print(filename)
    #filename = classes[i]
    #data = pickle.load(open(filename))
    data = pickle.load(open(filename,'rb'))
    target = [i] * len(data) #target sind quasi die label als integer, die postures, die werte in data sind die
    # angles der joints, gehören alle zu der Pose 'Knee' hier als [2] definiert
    
    return data, target
#print(target)
#load_pose_data(2)

In [99]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for i in range(11):
    #all_data.append(load_pose_data(i)[0]) #die 0 nimmt nur den 1. return wert
    #all_target.append(load_pose_data(i)[1]) # append erzeugt array in array
    all_data += load_pose_data(i)[0]
    all_target += load_pose_data(i)[1]

print('total number of data', len(all_data))
#print(all_data)
#print(all_target)

total number of data 222


In [100]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
#print(permutation)
#print(n_training_data)
#print(training_data)

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [101]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [102]:
# YOUR CODE HERE
#clf.fit(digits.data[:-500], digits.target[:-500]) 

all_data=np.array(all_data)
all_target=np.array(all_target)
tr_data = np.array(training_data)

#print(type(tr_data))
#print(type(tr_data[1]))

clf.fit(all_data[tr_data],all_target[tr_data]) #für indexing muss das index array ein np.array sein
# und auch das 'äußere array'

#print(all_data[tr_data)
#print(tr_data)
#print(all_data[-1])

SVC(C=100.0, gamma=0.001)

### predicting

In [103]:
#print(all_data)
#all_data2 = all_data[:,np.newaxis]
#for i in range(len(all_data)):
    #all_data[i]=all_data[i][:,np.newaxis]
#print(shape(all_data2))
#print(shape(all_data2[1]))
#all_data.reshape(1,-1)
#print(all_data)
#print(all_data[-1])
clf.predict([all_data[-1]]), all_target[-1]

(array([10]), 10)

In [104]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [105]:
expected = []
predicted = []
# YOUR CODE HERE
index_test = permutation[n_training_data:]
pred_test = clf.predict(all_data[index_test])
target_test = all_target[index_test]

#print(index_test)
expected.extend(target_test)
predicted.extend(pred_test)

#print(expected)
#print(predicted)


evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00        18
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00         8

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[11  0  0  0  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0]
 [ 0  0  0  0  7  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0  0  0]
 

## 4. Evaluate on the test data

In [106]:
expected = []
predicted = []
# YOUR CODE HERE

index_test = permutation[n_training_data:]
pred_test = clf.predict(all_data[index_test])
target_test = all_target[index_test]

#print(index_test)
expected.extend(target_test)
predicted.extend(pred_test)

#print(expected)
#print(predicted)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00        18
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00         8

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[11  0  0  0  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0]
 [ 0  0  0  0  7  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [107]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb')) #hier muss 'wb' stehen statt nur 'w' , b für binary

Then, in the application we can load the trained classifier again.

In [108]:
clf2 = pickle.load(open(ROBOT_POSE_CLF,'rb')) #hier 'rb' statt gar nichts
clf2.predict([all_data[-1]]), all_target[-1] # hier wieder eckige klammern um all_data[-1] da es liste von arrays sein muss und nicht nur ein element

(array([10]), 10)